In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

from pandas_datareader import data as web 
from datetime import timedelta

pd.options.mode.chained_assignment = None 

# Get the data
# I kept my real and synthetic data in separate folders with the same file names, so uncomment here as appropriate

df_basic = pd.read_csv('./cleaned/synthetic/basic.csv')
df_other = pd.read_csv('./cleaned/synthetic/other.csv')
df_dividend = pd.read_csv('./cleaned/synthetic/dividend.csv')

# df_basic = pd.read_csv('./cleaned/real/basic.csv')
# df_other = pd.read_csv('./cleaned/real/other.csv')
# df_dividend = pd.read_csv('./cleaned/real/dividend.csv')

In [ ]:
# Format date fields
df_basic['Date'] = pd.to_datetime(df_basic['Date'])
df_other['Date'] = pd.to_datetime(df_other['Date'])
df_dividend['Date'] = pd.to_datetime(df_dividend['Date'])

minDate = min(df_basic["Date"].min(), df_other["Date"].min())

tickers_all = df_basic['Name'].unique()

In [ ]:
##############################
#  Account for stock splits  #
##############################
allSplits = []

for ticker in tickers_all:

    # Get the split and convert to a df
    splits = yf.Ticker(ticker).splits

    df_split = []

    # If there was a split, append the df to allSplits
    if len(splits) > 0:
        df_split = pd.DataFrame({'Date':splits.index, 'Split':splits.values})
        df_split["Name"] = ticker
        allSplits.append(df_split[df_split["Date"] >= minDate])

allSplits = pd.concat(allSplits)

# Iterate through splits and alter share counts as necessary
for index, row in allSplits.iterrows():
    df_basic.loc[(df_basic["Name"] == row["Name"]) & (df_basic["Date"] < row["Date"]), ["ShareCount"]] = (df_basic['ShareCount'] * row["Split"]).round(6)

# Split buys and sells into separate dataframes
df_buys = df_basic.loc[df_basic["Type"] == "BUY"]
df_sells = df_basic.loc[df_basic["Type"] == "SELL"]

# A list of all tickers with their currency
tickers_currency = df_buys[["Name", "Currency"]].drop_duplicates().reset_index(drop=True)

# A list of currently held tickers
tickers_current = getDayPortfolio()["Name"].explode().unique() 

In [ ]:
# Get a valid pandas date
def getDateFormat(day, month, year):
    return pd.to_datetime(f"{year}-{month}-{day}")

In [ ]:
# Get the shares held on a given day
def getDayPortfolio(day = pd.to_datetime("today")):

    # Get the buys and sells for each day before the given date
    df_buysInRange = df_buys.loc[df_basic["Date"] <= day, ["Date", "Name", "ShareCount"]]
    df_sellsInRange = df_sells.loc[df_basic["Date"] <= day, ["Date", "Name", "ShareCount"]]

    # Any missing sells, i.e. any stock we've bought but not sold any of, need to still be in the resulting 'sell' df
    df_emptySells = df_buysInRange.copy()
    df_emptySells["ShareCount"] = 0
    df_sellsInRange = pd.concat([df_sellsInRange, df_emptySells])

    # Aggregate to get total shares in and out
    sharesIn = df_buysInRange.groupby('Name').agg({'ShareCount': 'sum'})['ShareCount'].reset_index()
    sharesOut = df_sellsInRange.groupby('Name').agg({'ShareCount': 'sum'})['ShareCount'].reset_index()

    # Format the aggregations
    sharesIn.rename(columns={"ShareCount": "SharesIn"}, inplace=True)
    sharesOut.rename(columns={"ShareCount": "SharesOut"}, inplace=True)
    sharesOut.drop(columns="Name", inplace=True)

    # Calculate net shares
    sharesTotal = pd.concat([sharesIn, sharesOut], axis=1)
    sharesTotal["ShareCount"] = (sharesTotal["SharesIn"] - sharesTotal["SharesOut"]).round(5)

    # Format the portfolio
    portfolio = sharesTotal.loc[sharesTotal["ShareCount"] != 0]
    portfolio = portfolio.drop(columns=["SharesIn", "SharesOut"])

    return portfolio



In [ ]:
# #Split buys and sells into separate dataframes
# df_buys = df_basic.loc[df_basic["Type"] == "BUY"]
# df_sells = df_basic.loc[df_basic["Type"] == "SELL"]

# # A list of currently held tickers
# tickers_current = getDayPortfolio()["Name"].explode().unique() 

# # A list of all tickers with their currency
# tickers_currency = df_buys[["Name", "Currency"]].drop_duplicates().reset_index(drop=True)

In [ ]:
# Get any necessary exchange rates
df_exchange_USDGBP = web.DataReader("GBPUSD=X", 'yahoo', df_buys["Date"].min(), pd.to_datetime("today"))
df_exchange_USDGBP.drop(columns=["High", "Low", "Open", "Volume", "Adj Close"], inplace=True)
df_exchange_USDGBP = df_exchange_USDGBP.reset_index()

# Get the first and last date
currentDate_fx = df_exchange_USDGBP["Date"].min()
lastDate_fx = pd.to_datetime("today")
lastDate_fx = lastDate_fx.replace(hour=0, minute=0, second=0, microsecond=0)

# Iterate through each day
while currentDate_fx != lastDate_fx:
    dayCounter_fx = 1

    # If the current day doesn't have an exchnage rate entry
    while len(df_exchange_USDGBP.loc[df_exchange_USDGBP["Date"] == currentDate_fx]) == 0:

        # Get the previous days entry
        prevDate_fx = currentDate_fx - timedelta(days=dayCounter_fx)
        df_previousFx = df_exchange_USDGBP.loc[df_exchange_USDGBP["Date"] == prevDate_fx]
        df_previousFx["Date"] = currentDate_fx

        # Add this entry to the exchange rate df
        df_exchange_USDGBP = df_exchange_USDGBP.append(df_previousFx, ignore_index=True)
        dayCounter_fx += 1

    currentDate_fx = currentDate_fx + timedelta(days=1)

# Rename the Close column to avoid confusion in the next cell...
df_exchange_USDGBP = df_exchange_USDGBP.rename(columns={"Close": "Rate"})


In [ ]:
# Import all closing prices for each ticker for the required date ranges
df_closing_all = []

for ticker in tickers_all:

    df_closing_ticker = []

    # Get the first transaction date for the ticker
    date_first = pd.to_datetime("today")
    date_first = df_buys.loc[df_buys["Name"] == ticker]["Date"].min()

    # Get the last transaction date for the ticker, or the current date if it's still being held
    date_last = pd.to_datetime("today")
    if ticker not in tickers_current:
        date_last = df_sells.loc[df_sells["Name"] == ticker]["Date"].max()

    # Get the close prices for each day
    # Yahoo has missing data for INRG. This isn't ideal but it's the best way to fix it - will need to manually download any mising data in future
    # TODO: think more carefully about how to deal with missing data - perhaps try multiple other sources?
    if ticker == "INRG.L":
        df_closing_ticker = pd.read_csv('./inputs/inrg.csv')
        df_closing_ticker["Date"] = pd.to_datetime(df_closing_ticker["Date"])
    else:
        df_closing_ticker = web.DataReader(ticker, 'yahoo', date_first, date_last)    

    dayCounter = 1
    
    while (date_first - df_closing_ticker.index.min())/np.timedelta64(1, 'D') < 0:
        df_closing_ticker = web.DataReader(ticker, 'yahoo', date_first - timedelta(days=dayCounter), date_last)  
        dayCounter += 1 

    # Format the df
    df_closing_ticker.drop(columns=["High", "Low", "Open", "Volume", "Adj Close"], errors="ignore", inplace=True)
    df_closing_ticker["Name"] = ticker

    # Get the currency for the current ticker
    tickerCurrency = tickers_currency.loc[tickers_currency["Name"] == ticker, "Currency"].values[0]

    # If it's USD, divide by the exchange rate for the relevant day
    if tickerCurrency == "USD":
        df_closing_ticker = pd.merge(df_closing_ticker, df_exchange_USDGBP, how="left", on="Date")
        df_closing_ticker["Close"] = df_closing_ticker["Close"]/df_closing_ticker["Rate"]
        df_closing_ticker.drop(columns=["Rate"], inplace=True)

    # If it's GBp, divide by 100 to convert to £
    elif tickerCurrency == "GBP":
        df_closing_ticker["Close"] = df_closing_ticker["Close"]/100
    else:
        print("ERROR: Unidentified Currency - ", ticker)
    
    df_closing_all.append(df_closing_ticker.reset_index())

# Concatenate all data
df_closing_all = pd.concat(df_closing_all)
df_closing_all.drop(columns=["index"], inplace=True)


In [ ]:
# Get daily portfolio value
dates = []
values = []

currentDate = df_buys["Date"].min()

endDate = pd.to_datetime("today")
endDate = endDate.replace(hour=0, minute=0, second=0, microsecond=0)

# Move day-by-day from the first date to now
while currentDate != endDate:
    # Get the portoflio of holdings for this date
    df_portfolio = getDayPortfolio(currentDate)

    # Get the closing prices for this day
    df_relevantClosing = df_closing_all.loc[df_closing_all["Date"] == currentDate]

    # Get all tickers for this day
    tickers = df_portfolio["Name"].values.tolist()

    # Loop through the tickers that we havent managed to extract closing data from, and keep checking the previous day until we get a value
    for t in tickers:        
        dayCounter = 1
        while t not in df_relevantClosing.values:
            prevDay = currentDate - timedelta(days=dayCounter)
            df_previousClosing = df_closing_all.loc[(df_closing_all["Date"] == prevDay) & (df_closing_all["Name"] == t)]
            df_relevantClosing = df_relevantClosing.append(df_previousClosing, ignore_index=True)
            dayCounter += 1

    # Merge these together, and calculate the value of each stock    
    df_portfolio = pd.merge(df_portfolio, df_relevantClosing, on="Name")
    df_portfolio["Value"] = df_portfolio["ShareCount"] * df_portfolio["Close"]
    dates.append(currentDate)
    values.append(df_portfolio["Value"].sum())

    currentDate = currentDate + timedelta(days=1)
        
df_dailyValue = pd.DataFrame(list(zip(dates, values)),columns =['Date', 'Value'])
df_dailyValue = df_dailyValue[df_dailyValue["Value"] != 0]

In [ ]:
# Generate a daily summary table using the 'other' dataframe initially
df_dailyActivity = pd.pivot_table(
    df_other, 
    values="Value", 
    index="Date", 
    columns="Type", 
    aggfunc=np.sum).fillna(0)

# Convert buys and sells to same format as daily summary
df_basicPivot = pd.pivot_table(
    pd.concat([df_buys[["Date", "Type", "Value"]], df_sells[["Date", "Type", "Value"]]]), 
    values="Value", index="Date", 
    columns="Type", 
    aggfunc=np.sum
).fillna(0)

# pivot dividend table
df_diviPivot = pd.pivot_table(
    df_dividend, 
    values="Value", index="Date",  
    aggfunc=np.sum
).fillna(0)

# Concatenate daily summary with buys, sells, and dividends
df_dailyActivity = pd.concat([df_dailyActivity, df_basicPivot, df_diviPivot], axis=1).fillna(0)

# Format daily summary
df_dailyActivity = df_dailyActivity.rename(columns={"BUY": "Total Bought", "SELL": "Total Sold", "Value": "Dividend"})
df_dailyActivity['Net Income']= df_dailyActivity.sum(axis=1)
df_dailyActivity = df_dailyActivity.round(2)
df_dailyActivity = df_dailyActivity.reset_index()


In [ ]:
# Merge dailyValue with activity
df_dailyActivity = pd.merge(df_dailyValue, df_dailyActivity, how="outer", on="Date")
df_dailyActivity.fillna(0, inplace=True)

# Add summary columns
df_dailyActivity["Cum Buy"] = df_dailyActivity["Total Bought"].cumsum()
df_dailyActivity["Cum Sell"] = df_dailyActivity["Total Sold"].cumsum()
df_dailyActivity["Net Cum"] = df_dailyActivity["Cum Buy"]*-1 - df_dailyActivity["Cum Sell"]
df_dailyActivity["Gains"] = df_dailyActivity["Value"] - df_dailyActivity["Net Cum"]

# Format
df_dailyActivity.drop(columns=["Cum Buy", "Cum Sell", "Net Cum"], inplace=True)
df_dailyActivity = df_dailyActivity.round(2)

In [ ]:
# Calculate rolling avgs
df_rolling = df_dailyActivity[["Date", "Value"]]

rolling7 = df_rolling.rolling(7).mean()
rolling30 = df_rolling.rolling(30).mean()
rolling60 = df_rolling.rolling(60).mean()

df_rolling["7Day"] = rolling7
df_rolling["30Day"] = rolling30
df_rolling["60Day"] = rolling60


In [ ]:
def plotPortfolio(graph, iDate, iValue, df = df_dailyValue, c = 'green', start = df_dailyValue["Date"].min(), end = df_dailyValue["Date"].max()):

    # Get the date range
    df_sub = df.loc[(df["Date"] >= start) & (df["Date"] <= end)]

    # Convert to Numpy array
    np_dailyValue = df_sub.to_numpy()
            
    # Define the plot line
    graph.plot(np_dailyValue[:,iDate], np_dailyValue[:,iValue], color=c)

In [ ]:
# Generate monthly summary

# Get, format, and summarise the 'total' columns
df_monthSum_Totals = df_dailyActivity[["Date", "Total Bought", "Total Sold"]]
df_monthSum_Totals["Total Bought"] = df_dailyActivity["Total Bought"]*-1
df_monthSum_Totals.set_index('Date', inplace=True)
df_monthSum_Totals = df_monthSum_Totals.resample('MS').sum()

# Get and summarise the 'value' columns
df_monthSum_Values = df_dailyActivity[["Date", "Value"]]
df_monthSum_Values.set_index('Date', inplace=True)
df_monthSum_Values = df_monthSum_Values.resample('MS').mean()

# Merge both summaries
df_monthSum = pd.merge(df_monthSum_Totals,df_monthSum_Values, on="Date")

# Format the resulting dataframe
df_monthSum.rename(columns={"Value": "Avg. Value"}, inplace=True)
df_monthSum.reset_index(inplace=True)
df_monthSum = df_monthSum.round(2)


In [ ]:
# Generate company summary

sum_trades = []
amount_bought = []
quant_bought = []
amount_sold = []
quant_sold = []
quant_current = []
value_current = []

for t in tickers_all:

    # Sum the buys and sells (not subtract, as buys are negative)
    sum_trades.append(len(df_buys.loc[df_buys["Name"] == t]) + len(df_sells.loc[df_sells["Name"] == t]))

    # Sum the # bought and sold
    quant_bought.append(df_buys.loc[df_buys["Name"] == t, "ShareCount"].sum().round(3))
    quant_sold.append(df_sells.loc[df_sells["Name"] == t, "ShareCount"].sum().round(3))

    # Sum the £ bought and sold
    amount_bought.append(df_buys.loc[df_buys["Name"] == t, "Value"].sum()*-1)
    amount_sold.append(df_sells.loc[df_sells["Name"] == t, "Value"].sum())
    
    # Work out how many shares we're currently holding
    holding = df_buys.loc[df_buys["Name"] == t, "ShareCount"].sum().round(3) - df_sells.loc[df_sells["Name"] == t, "ShareCount"].sum().round(3)

    # Work out the value of what we're currently holding
    close = df_closing_all.loc[df_closing_all["Name"] == t]
    quant_current.append(holding)
    value_current.append(close.loc[close['Date'].idxmax()]["Close"]*holding)

# Combine all these lists into a single dataframe
df_tickerSum = pd.DataFrame({
    'Ticker': tickers_all, 
    'Vol': sum_trades, 
    '# Bought': quant_bought, 
    '# Sold': quant_sold, 
    "£ Spent": amount_bought,
    "£ Sold": amount_sold,
    "# Holding": quant_current,
    "Value": value_current
})

In [ ]:
# Plots
fig = plt.figure(figsize=(12,8),dpi=100)
graph = fig.add_axes([0,0,1,1])
graph.xaxis.set_major_locator(plt.MaxNLocator(8))
graph.grid(True, color='0.6', dashes=(5, 2, 1, 2))

plotPortfolio(graph, 0,1, df_rolling, '#4385e0') # Daily Value
# plotPortfolio(graph, 0,2, df_rolling, '#6cc460') # 7-day rolling avg
# plotPortfolio(graph, 0,3, df_rolling, '#c99232') # 30-day rolling avg
plotPortfolio(graph, 0,4, df_rolling, '#d6493a') # 60-day rolling avg
print("Plotting daily + 60-day rolling avg")

In [ ]:
print("\n-- Overall Summary --")

firstDate = df_dailyActivity["Date"].min().strftime('%d/%m/%Y')
lastDate =  df_dailyActivity["Date"].max().strftime('%d/%m/%Y')
length_years = int(len(df_monthSum)/12)
length_months = len(df_monthSum)%12
print("You've been investing for {0} year(s) and {1} month(s), between {2} and {3}".format(length_years, length_months, firstDate, lastDate))

sum_trades = len(df_buys) + len(df_sells)
sum_buys = len(df_buys)
sum_sells = len(df_sells)
print("You made a total of {0} trades ({1} buys/{2} sells)".format(sum_trades, sum_buys, sum_sells))

sum_deposit = df_dailyActivity["Deposit"].astype(int).sum() - df_dailyActivity["Net Income"].sum()
print("You invested a total of £{0}".format(sum_deposit))

mean_buy = (df_buys["Value"].mean()*-1).round(2)
print("Your mean investment was £{}".format(mean_buy))

current_value = df_dailyActivity.loc[df_dailyActivity['Date'].idxmax()]["Value"].round(2)
increase = (current_value - sum_deposit).round(2)
increase_per = ((increase/sum_deposit)*100).round(1)
print("Your current portfolio is worth £{0}, this is an increase of £{1} ({2}%)".format(current_value, increase, increase_per))

print("\n-- Monthly Summary --")

maxBuy_month = df_monthSum.loc[df_monthSum['Total Bought'].idxmax()]["Date"]
maxBuy_amount = df_monthSum["Total Bought"].max().round(2)
print("You spent the most in {0} (£{1})".format(maxBuy_month, maxBuy_amount))

maxSell_month = df_monthSum.loc[df_monthSum['Total Sold'].idxmax()]["Date"]
maxSell_amount = df_monthSum["Total Sold"].max().round(2)
print("You sold the most in {0} (£{1})".format(maxSell_month, maxSell_amount))

maxValue_month = df_monthSum.loc[df_monthSum['Avg. Value'].idxmax()]["Date"]
maxValue_amount = df_monthSum["Avg. Value"].max().round(2)
print("Your highest-value month was {0} (£{1})\n".format(maxValue_month, maxValue_amount))

print(df_monthSum)

print("\n-- Ticker Summary --")
print(df_tickerSum)

In [ ]:
# Export all data
df_dailyActivity.to_csv('./outputs/dailyActivity.csv', index=False)
df_closing_all.to_csv('./outputs/dailyClose.csv', index=False)
df_rolling.to_csv('./outputs/valueRolling.csv', index=False)
df_monthSum.to_csv('./outputs/monthSum.csv', index=False)
df_tickerSum.to_csv('./outputs/tickerSum.csv', index=False)